In [ ]:
import bs4
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 드라이버 위치 설정
driver_loc = "./chromedriver.exe"
# 드라이버 옵션 설정

options = webdriver.ChromeOptions()

options.add_argument("window-size=2580x1440") # 파이썬이 크롬창을 띄울 것인데 창 크기 선택

# 웹 드라이버 정의
driver = webdriver.Chrome(driver_loc, options=options)

# 에누리 웹페이지 이동
# URL 정의
smartwatchUrl = 'http://www.enuri.com/'

driver.maximize_window()

driver.get(smartwatchUrl)

time.sleep(1.5)
driver.switch_to_default_content # 처음 iframe

tag = driver.find_element_by_xpath('//*[@id="gnbMenu"]/li[4]/a/em')
tag.click()

tag_main =driver.find_element_by_xpath('//*[@id="first_depth3"]/li[7]/p') 
tag_sub = driver.find_element_by_xpath('//*[@id="first_depth3"]/li[7]/div/ul[2]/li[1]/a')

ActionChains(driver).move_to_element(tag_main).click(tag_sub).perform()
time.sleep(0.5)

# 가격비교 이동
compareXpath = '//*[@id="tabsDiv"]/li[2]/a'
compareMenu = driver.find_element_by_xpath(compareXpath)
compareMenu.click()
time.sleep(0.5)

# 90개씩 보기
viewXpath = '//*[@id="pageGapSelDiv"]/a'
viewMenu = driver.find_element_by_xpath(viewXpath)
viewMenu.click()
time.sleep(0.5)

ninetyXpath = '//*[@id="pageGapSelDiv"]/ul/li[3]/a'
ninetyMenu = driver.find_element_by_xpath(ninetyXpath)
ninetyMenu.click()

try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "detailMultiLink prodName"))
    )
except Exception as e:
    print(e)

rowList = []
titleList = []
for i in range(0, 999):
    try:
        if i > 0:
            # 다음 페이지 이동
            url = 'http://www.enuri.com/list.jsp?cate=03181105&tabType=1&page=' + str(i + 1)
            driver.get(url)
            time.sleep(5)
        
        data = driver.page_source
        bs = bs4.BeautifulSoup(data, 'html.parser')

        ulTag = bs.find(name = 'ul',
                       attrs = {'id': 'listBodyDiv'})
        liTags = ulTag.findAll(name = 'li',
                              attrs = {'class': 'prodItem wide'})
        print(len(liTags))
        # //*[@id="modelno_37828916"]/div[1]/strong/a
        # //*[@id="modelno_34805103"]/div[1]/strong/a
        # //*[@id="modelno_39729557"]/div[1]/strong/a
        
        # xpath이용 -> //*[@id="xxxx"]/div[1]/strong/a ,xxxx부분에 상품 id
        for j in range(len(liTags)):
            if not (i == 0 & (j == 77 | j == 88)):
                productNum = liTags[j].attrs['id']
                print(productNum,type(productNum))
                productXpath = '//*[@id="{}"]/div[1]/strong/a'.format(productNum)
                driver.find_element_by_xpath(productXpath).click()
                time.sleep(5)

                second_tab = driver.window_handles[-1]
                driver.switch_to.window(second_tab)

                data2 = driver.page_source
                smartwatchBs = bs4.BeautifulSoup(data2, 'html.parser')

                # 최저가 추출

                tableTag = smartwatchBs.find('table', {'class':'malllow__list'})
                tbodyTag = tableTag.find('tbody', {'id':'vip_malllow_item_list_id'})
                tdTag = tbodyTag.find('td', {'class':'malllow__item price'})
                lowPrice = tdTag.find('strong').text.replace(',','')
                lowPriceTitle = tdTag.find('span').text

                columnList = []

                titleList.append(lowPriceTitle)
                columnList.append(lowPrice)

                # 각종 특성 추출
                divTag = smartwatchBs.find('div', {'id':'enuri_spec_table'})
                tbodyTag = divTag.find('tbody')
                thTags = tbodyTag.findAll('th', {'scope':'row'})
                tdTags = tbodyTag.findAll('td')

                for k in range(len(thTags)):
                    if thTags[k].text != " ":
                        if i == 0 & j == 0:
                            titleList.append(thTags[k].text.strip())

                        columnList.append(tdTags[k].text.strip())

                print(columnList)
                rowList.append(columnList)
                columnList = []
                # 컬럼명 및 스펙 정보 추출

#                 print(type(data))   
                driver.close()

                first_tab = driver.window_handles[0]
                driver.switch_to.window(first_tab)

                time.sleep(2)
    except:
        break
        #     first_tab = driver.window_handles[0]
        #     driver.switch_to.window(window_name=second_tab )


        # item1Xpath = '#modelno_39729557 > div.sp_title > strong > a'
        # item1Menu = driver.find_element_by_css_selector(item1Xpath)
        # item1Menu.click()
        # time.sleep(2)

Message: 

89
89
modelno_44586517 <class 'str'>
['39900', '스마트워치', '1.3"', '240x240']
<class 'str'>
modelno_32742163 <class 'str'>
['57810', '스마트워치', '0.96"', '80x160', 'LCD', '', '']
<class 'str'>
modelno_43504827 <class 'str'>
['221930', '스마트워치', 'LTE', '무선랜(WiFi)', '368x448', 'AM-OLED', '블루투스5.0']
<class 'str'>
modelno_49173320 <class 'str'>
['89000', '스마트워치', '밴드교체형', '1.22"', '240x240', '', '']
<class 'str'>
modelno_27181635 <class 'str'>
['150220', '스마트워치', '밴드교체형', '터치스크린', '블루투스', '', '']
<class 'str'>
modelno_41662337 <class 'str'>
['685020', '스마트워치', '밴드교체형', '무선랜(WiFi)', '368x448', 'OLED', '터치스크린', '블루투스5.0', '', '']
<class 'str'>
modelno_48882445 <class 'str'>
['667990', '스마트워치', '밴드교체형', '무선랜(WiFi)', '1.2"', '240x240', '블루투스']
<class 'str'>
modelno_49175901 <class 'str'>
['486000', '스마트워치', '밴드교체형', '무선랜(WiFi)', '368x448', 'OLED', '터치스크린', '블루투스5.0', '', '']
<class 'str'>
modelno_41662243 <class 'str'>
['528300', '스마트워치', '밴드교체형', '무선랜(WiFi)', '324x394', 'OLED', '블루투스5.0']